### Semi-autonomous Teleoperation Demo : Place task in `Office` scene

### Parse `Realistic Scene`

In [1]:
import mujoco,cv2,pyvista
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../../../')
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import sample_xyzs,rpy2r,r2rpy,r2quat,compute_view_params,get_interp_const_vel_traj, printmd
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


In [2]:
xml_path = '../../../asset/scene_realworld_wo_shelf.xml'
env = MuJoCoParserClass(name='Place task scene: Office table',rel_xml_path=xml_path,VERBOSE=False, MODE='window')
print(env.MODE)

# Move tables and robot base
env.model.body('base_table').pos = np.array([0,0,0])
env.model.body('avoiding_object_table').pos = np.array([0.38+0.45,0,0])
env.model.body('base').pos = np.array([0.18,0,0.79])
env.model.body('right_object_table').pos = np.array([-0.05,-0.80,0])
env.model.body('left_object_table').pos = np.array([-0.05,0.80,0])

# Place objects
tray_pos = np.array([ 0.9, 0.0, 0.8])
bowl_red_pos = np.array([ 0.9, 0.3, 0.8])
bowl_blue_pos = np.array([ 0.9, -0.3, 0.8])
env.model.joint(env.model.body('tray_gray').jntadr[0]).qpos0[:3] = tray_pos
env.model.joint(env.model.body('kitchen-bowl-red').jntadr[0]).qpos0[:3] = bowl_red_pos
env.model.joint(env.model.body('kitchen-bowl-blue').jntadr[0]).qpos0[:3] = bowl_blue_pos

env.model.joint(env.model.body('ycb-apple-2').jntadr[0]).qpos0[:3] = bowl_red_pos + np.array([0.0,0.0,0.02])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[:3] = tray_pos + np.array([0.0,0.03,0.2])
env.model.joint(env.model.body('ycb-banana-2').jntadr[0]).qpos0[3:] = r2quat(rpy2r(np.radians([0, 0, 90])))
env.model.joint(env.model.body('ycb-lemon-2').jntadr[0]).qpos0[:3] = tray_pos + np.array([-0.03,-0.07,0.02])
env.model.joint(env.model.body('ycb-orange-2').jntadr[0]).qpos0[:3] = bowl_blue_pos + np.array([0.0,0.0,0.02])

# Target objects
env.model.joint(env.model.body('ycb-apple').jntadr[0]).qpos0[:3] = np.array([0.1, -0.6, 0.8])
env.model.joint(env.model.body('ycb-lemon').jntadr[0]).qpos0[:3] = np.array([0.0, -0.6, 0.8])
env.model.joint(env.model.body('ycb-orange').jntadr[0]).qpos0[:3] = np.array([-0.1, -0.6, 0.8])


window


In [5]:

# Init viewer
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True, MODE='window')
env.update_viewer(azimuth=30,distance=3.0,elevation=-30,lookat=[1.0,0.0,0.71],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=False,jointlength=0.25,jointwidth=0.05,jointrgba=[0.2,0.6,0.8,0.6])

pcd_tick = 0

# Reset
env.reset()

# init_ur_q = np.array([np.deg2rad(-90), np.deg2rad(-130), np.deg2rad(120), np.deg2rad(100), np.deg2rad(45), np.deg2rad(-90)])
init_ur_q = np.array([np.deg2rad(-180), np.deg2rad(-30), np.deg2rad(-120), np.deg2rad(125), np.deg2rad(-90), np.deg2rad(0)])

env.forward(q=init_ur_q, joint_idxs=env.idxs_forward)
initial_xyz = env.get_p_body('tcp_link')
print(env.get_p_body('tcp_link'))
tick,max_sec = 0,1000

i = 0

while env.get_sim_time() <= max_sec:    
    env.step(ctrl=init_ur_q,ctrl_idxs=env.idxs_step)
    # env.model.geom_size[env.model.body('obj_box_03').geomadr[0]] += np.array([0.0001,0.0001,0.0001])
    if not env.is_viewer_alive(): break

    # Render
    if env.loop_every(HZ=20):
        # Compute some poses
        p_tcp,R_tcp = env.get_pR_body(body_name='tcp_link')
        p_cam,R_cam = env.get_pR_body(body_name='camera_center')
        p_base,R_base = env.get_pR_body(body_name='base')
        # Get PCD from a specific view
        p_ego  = p_cam
        p_trgt = p_cam + R_cam[:,2] # + np.array([0,0,-0.2])
        rgb_img,depth_img,pcd,xyz_img = env.get_egocentric_rgb_depth_pcd(
            p_ego=p_ego,p_trgt=p_trgt,rsz_rate=40,fovy=45,BACKUP_AND_RESTORE_VIEW=True)
        env.render(render_every=1)

# Close viewer
env.close_viewer()

[ 0.35 -0.13  1.16]
Pressed ESC
Quitting.


In [ ]:
env.reset()

# object position lists on the table
obj_pos_list_table = {}
obj_pos_list_table['kitchen-drainer'] = {'position':env.get_p_body('kitchen-drainer'), 'size':np.array([0.17,0.07,0.28])}
obj_pos_list_table['tray'] = {'position':env.get_p_body('tray'), 'size': np.array([0.11,0.15,0.01])}
obj_pos_list_table['mug_cup'] = {'position':env.get_p_body('mug_cup'), 'size': np.array([0.05,0.07,0.1])}
obj_pos_list_table['kitchen-plate-2'] = {'position':env.get_p_body('kitchen-plate-2'), 'size': np.array([0.1,0.1,0.01])}
obj_pos_list_table['kitchen-plate-3'] = {'position':env.get_p_body('kitchen-plate-3'), 'size': np.array([0.1,0.1,0.01])}

print(obj_pos_list_table)
print(list(obj_pos_list_table.keys()))

{'kitchen-drainer': {'position': array([0.9 , 0.35, 0.82]), 'size': array([0.17, 0.07, 0.28])}, 'tray': {'position': array([ 0.9, -0.3,  0.8]), 'size': array([0.11, 0.15, 0.01])}, 'mug_cup': {'position': array([ 0.9 , -0.3 ,  0.85]), 'size': array([0.05, 0.07, 0.1 ])}, 'kitchen-plate-2': {'position': array([0.9 , 0.29, 1.07]), 'size': array([0.1 , 0.1 , 0.01])}, 'kitchen-plate-3': {'position': array([0.9 , 0.23, 1.07]), 'size': array([0.1 , 0.1 , 0.01])}}
['kitchen-drainer', 'tray', 'mug_cup', 'kitchen-plate-2', 'kitchen-plate-3']
